# First look at genotyping error in CRAGIG Run1

I finally made a Genepop of the tags (not the SNPs) and was visually assessing the sequencing replicates, and noticed that they were actually pretty different. Bummer! Here, I'm measuring genotyping error. Ideally there'd be a way to treat a homozygote called as a different homozygote differently form a homozygote called as a heterozygote with one of the same alleles, but for now I'll try to just count both kinds.

Brent just stopped by and suggested the following:
- Test to see if places with more than two alleles is within a tag
- Test algorithm on samples extracted together

In [78]:
cd /mnt/hgfs/SHARED_FOLDER/WorkingFolder

/mnt/hgfs/SHARED_FOLDER/WorkingFolder


Read in file, get list of lists to turn into numpy array

In [79]:
input = open("cragigrun1_reps_genotypes.txt")
lines = input.readlines()
biglist = []
for line in lines:
    linelist = line.strip().split()
    biglist.append(linelist)

In [80]:
import numpy as np
myarray = np.array(biglist)

In [81]:
print myarray

[ ['FG100_A,', '0304', '0101', '0505', '0711', '0606', '0303', '0707', '0202', '0404', '0101', '0101', '0606', '0101', '0305', '0303', '0404', '0606', '0303', '0303', '0505', '0607', '0101', '0506', '0203', '0404', '0000', '1111', '1414', '0404', '1010', '0000', '0606', '0202', '0208', '0606', '0909', '0203', '0101', '0000', '0308', '0505', '0000', '0000', '1313', '0303', '0404', '1111', '1215', '0404', '0202', '0101', '0303', '0202', '1111', '0707', '1111', '0609', '0909', '3030', '0303', '0404', '0404', '0404', '0909', '0909', '0101', '1616', '0102', '0202', '0404', '0000', '0404', '0606', '0303', '0405', '0608', '0305', '0101', '0505', '0303', '0505', '0506', '1212', '0809', '0606', '0607', '0507', '0303', '0304', '0202', '0411', '0707', '0000', '0507', '1111', '0305', '0707', '0406', '0202', '0404', '0505', '0505', '1317', '0303', '0404', '0303', '0707', '0000', '0204', '1414', '0000', '0505', '1313', '0606', '0306', '1019', '1011', '0202', '0404', '0909', '0202', '0304', '0505', '

Slice array into couplets, named for replicates of sample

In [82]:
FG100 = myarray[0:2]
FG101 = myarray[2:4]
FG102 = myarray[4:6]
FG103 = myarray[6:8]
FG104 = myarray[8:10]
tags_header = myarray[10]

In [83]:
print FG100

[ ['FG100_A,', '0304', '0101', '0505', '0711', '0606', '0303', '0707', '0202', '0404', '0101', '0101', '0606', '0101', '0305', '0303', '0404', '0606', '0303', '0303', '0505', '0607', '0101', '0506', '0203', '0404', '0000', '1111', '1414', '0404', '1010', '0000', '0606', '0202', '0208', '0606', '0909', '0203', '0101', '0000', '0308', '0505', '0000', '0000', '1313', '0303', '0404', '1111', '1215', '0404', '0202', '0101', '0303', '0202', '1111', '0707', '1111', '0609', '0909', '3030', '0303', '0404', '0404', '0404', '0909', '0909', '0101', '1616', '0102', '0202', '0404', '0000', '0404', '0606', '0303', '0405', '0608', '0305', '0101', '0505', '0303', '0505', '0506', '1212', '0809', '0606', '0607', '0507', '0303', '0304', '0202', '0411', '0707', '0000', '0507', '1111', '0305', '0707', '0406', '0202', '0404', '0505', '0505', '1317', '0303', '0404', '0303', '0707', '0000', '0204', '1414', '0000', '0505', '1313', '0606', '0306', '1019', '1011', '0202', '0404', '0909', '0202', '0304', '0505', '

In [84]:
print tags_header

['5,6,13,16,20,22,27,28,31,35,49,50,57,62,63,71,75,77,79,80,84,86,89,92,93,94,95,96,100,101,111,112,114,117,119,122,123,125,127,128,131,134,138,140,147,148,182,183,190,193,196,199,201,206,219,220,222,223,224,226,230,234,238,243,244,245,248,251,253,255,256,259,262,268,278,283,285,291,303,304,305,306,308,309,311,312,317,319,323,327,333,336,337,341,342,343,350,352,356,357,360,367,369,370,371,376,377,379,380,382,387,391,392,395,398,400,401,405,413,436,439,447,448,450,455,460,461,465,467,471,473,476,480,482,485,486,488,499,510,512,513,514,517,522,523,524,526,528,532,534,542,545,549,555,558,561,563,566,571,572,582,586,588,600,606,607,610,614,615,617,624,625,626,628,631,633,636,641,651,654,657,659,662,672,676,678,681,682,697,698,700,703,705,708,714,717,719,724,735,741,751,754,757,762,767,776,778,779,790,793,795,797,802,803,806,807,811,817,819,820,822,825,827,828,829,830,839,842,845,848,849,851,859,860,865,866,867,870,881,883,884,889,891,893,899,902,903,904,908,909,910,914,915,922,923,930,932,

In [85]:
# get list of tags
tags_list = tags_header[0].strip().split(",")
print tags_list[0:9]

['5', '6', '13', '16', '20', '22', '27', '28', '31']


In [87]:
# break into replicate couplets, name by sample
list_of_arrays = []
list_of_arrays.append(FG100)
list_of_arrays.append(FG101)
list_of_arrays.append(FG102)
list_of_arrays.append(FG103)
list_of_arrays.append(FG104)

In [107]:
# initiate counts and lists to track genotyping error
match_genotypes_count = 0
match_genotypes_tags = []
partmismatch_genotypes_count = 0
partmismatch_genotypes_tags = []
totalmismatch_genotypes_count = 0
totalmismatch_genotypes_tags = []
nodata_mismatch_count = 0
nodata_mismatch_tags = []

In [108]:
num_tags = len(tags_list) # iterate over number of tags
print num_tags

7749


### Report genotype errors and matches by sample

I had 5 samples, each with 2 sequencing replicates. Here I'm comparing them, and reporting back how many had a perfect match, how many had one allele match, how many didn't match at all. Percents are over a total of genotypes excluding those with any missing data.

In [109]:
# sort genotype match/mismatch by sample
bysam_match_tags = []
bysam_partmismatch_tags = []
bysam_totalmismatch_tags = []
bysam_missing_tags = []

timescount = 1
for array in list_of_arrays:
    sam_match_genotypes_count = 0
    sam_match_genotypes_tags = []
    sam_partmismatch_genotypes_count = 0
    sam_partmismatch_genotypes_tags = []
    sam_totalmismatch_genotypes_count = 0
    sam_totalmismatch_genotypes_tags = []
    sam_nodata_mismatch_count = 0
    sam_nodata_mismatch_tags = []
    for i in range(1,num_tags):
        A_gen = array[0][i] # get genotypes
        B_gen = array[1][i]
        A_1 = A_gen[0:2] # pull out alleles
        A_2 = A_gen[2:4]
        B_1 = B_gen[0:2]
        B_2 = B_gen[2:4]
        A_list = [] # store alleles in lists
        A_list.append(A_1)
        A_list.append(A_2)
        B_list = []
        B_list.append(B_1)
        B_list.append(B_2)
        first_test_string = B_1 + B_2
        second_test_string = B_2 + B_1
        match_options = []
        match_options.append(first_test_string)
        match_options.append(second_test_string)
        if (A_gen == "0000" or B_gen == "0000") or (A_gen == "0000" and B_gen == "0000"):
            nodata_mismatch_count += 1
            nodata_mismatch_tags.append(tags_list[i])
            sam_nodata_mismatch_count +=1
            sam_nodata_mismatch_tags.append(tags_list[i])
        elif A_gen in match_options: # if perfect match
            match_genotypes_count += 1
            match_genotypes_tags.append(tags_list[i])
            sam_match_genotypes_count += 1
            sam_match_genotypes_tags.append(tags_list[i])
        elif A_1 in B_list or A_2 in B_list: # if one allele is a match
            partmismatch_genotypes_count += 1
            partmismatch_genotypes_tags.append(tags_list[i])
            sam_partmismatch_genotypes_count += 1
            sam_partmismatch_genotypes_tags.append(tags_list[i])
        elif A_1 not in B_list or A_2 not in B_list: # if not a match at all
            totalmismatch_genotypes_count += 1
            totalmismatch_genotypes_tags.append(tags_list[i])
            sam_totalmismatch_genotypes_count += 1
            sam_totalmismatch_genotypes_tags.append(tags_list[i])
        else:
            print "Your loop may not be fully working." 
            print "You thought there were three possible options, but there were more than three. Check!"
        
    bysam_missing_tags.append(sam_nodata_mismatch_tags)
    bysam_match_tags.append(sam_match_genotypes_tags)
    bysam_partmismatch_tags.append(sam_partmismatch_genotypes_tags)
    bysam_totalmismatch_tags.append(sam_totalmismatch_genotypes_tags)

    print "\nSample number " + str(timescount) + " :"
    print "Genotypes with perfect match: " + str(sam_match_genotypes_count) + " : "
    print "Genotypes with partial match: " + str(sam_partmismatch_genotypes_count) + " : "
    print "Genotypes with total mismatch: " + str(sam_totalmismatch_genotypes_count) + " : "
    print "Genotypes with mismatch due to missing data: " + str(sam_nodata_mismatch_count) + " : "
        
    timescount +=1


Sample number 1 :
Genotypes with perfect match: 2408 : 
Genotypes with partial match: 2492 : 
Genotypes with total mismatch: 1602 : 
Genotypes with mismatch due to missing data: 1246 : 

Sample number 2 :
Genotypes with perfect match: 2493 : 
Genotypes with partial match: 2507 : 
Genotypes with total mismatch: 1769 : 
Genotypes with mismatch due to missing data: 979 : 

Sample number 3 :
Genotypes with perfect match: 2508 : 
Genotypes with partial match: 2577 : 
Genotypes with total mismatch: 1775 : 
Genotypes with mismatch due to missing data: 888 : 

Sample number 4 :
Genotypes with perfect match: 2409 : 
Genotypes with partial match: 2580 : 
Genotypes with total mismatch: 1825 : 
Genotypes with mismatch due to missing data: 934 : 

Sample number 5 :
Genotypes with perfect match: 2388 : 
Genotypes with partial match: 2554 : 
Genotypes with total mismatch: 1769 : 
Genotypes with mismatch due to missing data: 1037 : 


In [128]:
from __future__ import division

total_excl_mv = match_genotypes_count + partmismatch_genotypes_count + totalmismatch_genotypes_count
print "Total genotypes excluding those with missing data : " + str(total_excl_mv)
print "\nTotal genotypes with perfect match: " + str(match_genotypes_count) + " : " + str((float(match_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with partial match: " + str(partmismatch_genotypes_count) + " : " + str((float(partmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with total mismatch: " + str(totalmismatch_genotypes_count) + " : " + str((float(totalmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "\nTotal genotypes with mismatch due to missing data: " + str(nodata_mismatch_count)

Total genotypes excluding those with missing data : 33656

Total genotypes with perfect match: 12206 : 36.26%
Total genotypes with partial match: 12710 : 37.76%
Total genotypes with total mismatch: 8740 : 25.96%

Total genotypes with mismatch due to missing data: 5084


Bummer! That's a ton of errors. These "totals" are across all five samples. Above, you can see the results split by sample. Although I didn't calculate percent, you can see they're pretty similar.

### Checking for sample overlap between tags that did show a perfect match within a sample's replicates

Next check - are the tags that have a perfect match consistent across samples? 

Here's the first 20 tags within each sample's list of tags that had perfect match between replicates:

In [104]:
print bysam_match_tags[0][0:20]
print bysam_match_tags[1][0:20]
print bysam_match_tags[2][0:20]
print bysam_match_tags[3][0:20]
print bysam_match_tags[4][0:20]

['13', '16', '22', '27', '28', '31', '35', '49', '50', '75', '77', '79', '84', '86', '89', '94', '100', '114', '117', '122']
['13', '16', '22', '27', '31', '35', '49', '50', '75', '79', '84', '89', '94', '100', '114', '117', '122', '128', '147', '193']
['6', '16', '20', '22', '27', '31', '35', '49', '50', '75', '79', '84', '89', '92', '94', '95', '96', '100', '101', '111']
['6', '16', '20', '27', '31', '49', '50', '71', '75', '89', '93', '94', '95', '96', '100', '111', '117', '122', '131', '140']
['13', '20', '27', '49', '50', '57', '75', '89', '93', '94', '96', '111', '117', '122', '134', '148', '199', '201', '206', '219']


From first glance, it does seem like there's some significant (but definitely not complete) overlap of tags between samples that had perfect genotype match within replicates. A closer look at that overlap...

Getting percent of tags that match in all five samples:

In [137]:
matching = len(set(bysam_match_tags[0]) and set(bysam_match_tags[1]) and set(bysam_match_tags[2]) and set(bysam_match_tags[3]) and set(bysam_match_tags[4]))
total = len(set(match_genotypes_tags))
perc_same = (float(matching)/float(total))*100
print str(perc_same)[0:5] + "% of tags that showed a perfect match in at least one set of replicates, also perfectly matched in all 5 sets of replicates."


46.01% of tags that showed a perfect match in at least one set of replicates, also perfectly matched in all 5 sets of replicates.


Seems like they're just as likely to match as not match, so that's not super helpful.